### 🔄 Data Transformation
In this notebook, we will carry out a comprehensive data transformation process to ensure that the data is structured and usable for subsequent analysis or reporting.

We will load the environment variables from the .env file, which contains essential configurations such as database credentials. Then, we'll get the working directory from these variables and add it to the system path to ensure that the project's modules can be imported correctly.

In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

print('Workdir: ', work_dir)


Workdir:  C:/Users/Administrador/Desktop/Workshops ETL/Workshop_ETL


Next, we will import the necessary modules and classes for the rest of the notebook.



In [2]:
from src.db_connection import build_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect
from src.model import CandidatesTransformed
from sqlalchemy.exc import SQLAlchemyError
from src.transform_data import DataTransformer


We will now establish a connection to the PostgreSQL database using the build_engine function.

A SQLAlchemy session will be created using the database engine established in the previous step. This session will be used for performing read and write operations on the database.

In [3]:
engine = build_engine()
Session = sessionmaker(bind=engine)
session = Session()


Successfully connected to the database postgres!


We will check if the CandidatesTransformed table exists in the database. If it does, the table will be dropped, and a new table will be created. This ensures that the table is up-to-date and ready to receive new data.



In [4]:
try:
    inspector = inspect(engine)

    if inspector.has_table('CandidatesTransformed'):
        try:
            CandidatesTransformed.__table__.drop(engine)
        except SQLAlchemyError as e:
            print(f"Error dropping table: {e}")
            raise
    try:
        CandidatesTransformed.__table__.create(engine)
        print("Table creation was successful.")
    except SQLAlchemyError as e:
        print(f"Error creating table: {e}")
        raise

except SQLAlchemyError as error:
    print(f"An error occurred: {error}")


Table creation was successful.


Now it's time to perform the necessary data transformations. These transformations will include:

- Generating an ID column.
- Renaming columns for consistency.
- Adding the Hired column based on the candidate's scores.
- Categorizing technologies.
- Saving the transformed data back to the database.

In [5]:
try:
    transformer = DataTransformer('../data/candidates.csv')
    
    # Standardizing and transforming data
    transformer.generate_ids()
    transformer.standardize_column_names()
    transformer.calculate_hiring_status()
    transformer.categorize_technology()
    
    # Saving transformed data to the database
    transformer.save_transformed_data(output_path='../data/candidates_transformed.csv')
    transformer.data.to_sql('CandidatesTransformed', engine, if_exists='append', index=False)
    print("Data transformation and upload were successful.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if session:
        session.close()


Data transformation and upload were successful.


### ✅ Conclusion
In this notebook, we successfully transformed the raw data by standardizing column names, generating unique IDs, adding a 'Hired' column, categorizing technologies, and uploading the transformed data to the PostgreSQL database.







